In [3]:
import pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
from datetime import datetime 

from keras.models import Model 
from keras.layers import Input, Embedding, Dot, Add, Flatten 
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

ModuleNotFoundError: No module named 'tensorflow'

(10000, 2000)